# Классификация тональности текста

## Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества F1 не меньше 0.75.

# План работы
- Анализ и обработка данных
- Обучение
- Вывод

# 1. Анализ и обработка данных

In [ ]:
! pip install nltk

In [ ]:
! pip install wordcloud

In [ ]:
import pandas as pd
import re
import numpy as np
import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from concurrent.futures import ThreadPoolExecutor
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
import lightgbm as lgb
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from nltk.corpus import wordnet
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('omw')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\www\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\www\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\www\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\www\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\www\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
except:
    df = pd.read_csv('C:/Users/www/toxic_comments.csv', index_col=0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [ ]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
df['text']

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159446    ":::::And for the second time of asking, when ...
159447    You should be ashamed of yourself \n\nThat is ...
159448    Spitzer \n\nUmm, theres no actual article for ...
159449    And it looks like it was actually you who put ...
159450    "\nAnd ... I really don't think you understand...
Name: text, Length: 159292, dtype: object

Проведу обработку данных с помощью функции

In [ ]:
# Функция для очистки текста
def clean_text(text):
    try:
        # Удаляем символы новой строки
        text = re.sub(r"(?:\n|\r)", " ", text)
        # Удаляем все символы, кроме буквенных, и удаляем начальные и конечные пробелы
        text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
        # Преобразуем в нижний регистр
        text = text.lower()
        # Удаляем стоп-слова и проводим лемматизацию
        nltk.download('stopwords', quiet=True)
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        words = re.findall(r'\b\w+\b', text)
        with ThreadPoolExecutor() as executor:
            words = list(executor.map(lemmatizer.lemmatize, ((w, get_wordnet_pos(w)) for w in words if w not in stop_words)))
        text = ' '.join(words)
        # Проверяем, не является ли текст пустым
        if not text:
            raise ValueError('Пустой текст после очистки')
        return text
    except Exception as e:
        print(f'Ошибка при очистке текста: {e}')
# Загрузим WordNetLemmatizer для лемматизации слов
lemmatizer = WordNetLemmatizer()

# Функция для определения части речи слова
def get_wordnet_pos(word):
    """
    Определение части речи с помощью библиотеки nltk и WordNet
    """
    # Определение тега части речи с помощью метода pos_tag
    # из библиотеки nltk и присваивание тега переменной tag
    tag = nltk.pos_tag([word])[0][1][0].upper()

    # Создание словаря соответствий между тегами частей речи
    # и соответствующими значениями частей речи из WordNet
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    # Возвращение соответствующей части речи из словаря,
    # либо по умолчанию NOUN
    return tag_dict.get(tag, wordnet.NOUN)

# Функция для лемматизации текста
def lemm_(text):
    """
    Лемматизация текста с помощью библиотеки nltk и WordNetLemmatizer
    """
    # Токенизация текста - разделение на слова
    tokens = nltk.word_tokenize(text)

    # Лемматизация каждого слова в тексте с помощью
    # функции get_wordnet_pos для определения части речи слова
    lemmatized_tokens = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]

    # Склеивание лемматизированных слов обратно в строку
    lemmatized_output = ' '.join(lemmatized_tokens)

    # Возвращение лемматизированной строки
    return lemmatized_output

# Пример использования функции для датафрейма df с колонкой text
# Вывод колонки text до применения функции
print('было')
print(df['text'])

# Применение функции lemm_ к каждой строке колонки text
# с помощью метода apply() и сохранение результата в новой колонке lemm_text
df['text'] = df['text'].apply(lemm_)

# Вывод колонки lemm_text после применения функции
print('стало после токенизации и лемматизации')
print(df['text'])

было
0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159446    ":::::And for the second time of asking, when ...
159447    You should be ashamed of yourself \n\nThat is ...
159448    Spitzer \n\nUmm, theres no actual article for ...
159449    And it looks like it was actually you who put ...
159450    "\nAnd ... I really don't think you understand...
Name: text, Length: 159292, dtype: object


KeyboardInterrupt: 

In [ ]:
df['text']

# 2 Обучение

Разделю выборки на трейн и тест

In [ ]:
# Определяем целевую переменную и признаки
target = df['toxic'].values
features = df['text']

# Разбиваем на тренировочную и тестовую выборки
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.1, random_state=44)

# Выводим размеры каждой выборки
print(f"Размер тренировочной выборки: {features_train.shape[0]}")
print(f"Размер тестовой выборки: {features_test.shape[0]}")

In [ ]:
# Загружаем стоп-слова английского языка из библиотеки NLTK
stop_words = set(stopwords.words('english'))

# Создаем объект TfidfVectorizer, указывая список стоп-слов
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

# Вычисляем TF-IDF для тренировочной и тестовой выборок
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

# Выводим размеры каждой выборки
print(f"Размер тренировочной выборки: {tf_idf_train.shape[0]}")
print(f"Размер тестовой выборки: {tf_idf_test.shape[0]}")

Подберу параметры

In [ ]:
%%time
# Создаем пайплайн
pipe = Pipeline([
    (
        ('model', LogisticRegression(random_state=44, solver='liblinear', max_iter=200))
    )
])

# Определяем сетку гиперпараметров для поиска
param_grid = {
    'model__penalty': ['l1', 'l2'],
    'model__C': [1, 4, 7, 10, 13],
}

# Создаем объект GridSearchCV и запускаем поиск
grid_search_lr = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring='f1',
    cv=3,
    verbose=True,
    n_jobs=-1
)

# Обучаем модель на обучающей выборке
grid_search_lr.fit(tf_idf_train, target_train)

# Выводим лучшие параметры и лучший результат
print('LogisticRegression Best parameters:', grid_search_lr.best_params_)
print('LogisticRegression Best F1-score:', grid_search_lr.best_score_)

In [ ]:
%%time
# Создаем пайплайн
pipe = Pipeline([
    (
        ('model', lgb.LGBMClassifier(random_state=44, num_threads=2))
    )
])

# Определяем сетку гиперпараметров для поиска
param_grid = {
    'model__boosting_type': ['gbdt', 'dart'],
    'model__num_leaves': [10, 20, 30],
    'model__max_depth': [3, 5, 7],
    'model__learning_rate': [0.1, 0.05, 0.01],
}

# Создаем объект GridSearchCV и запускаем поиск
grid_search = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring='f1',
    cv=3,
    verbose=True,
    n_jobs=-1
)

# Обучаем модель на обучающей выборке
grid_search.fit(tf_idf_train, target_train)

# Выводим лучшие параметры и лучший результат
print('lgb Best parameters:', grid_search.best_params_)
print('lgb Best F1-score:', grid_search.best_score_)

проверю на тестовых данных модель регрессии

In [ ]:
# Предсказываем классы на тестовой выборке
test_pred = grid_search_lr.predict(tf_idf_test)

# Вычисляем F1-оценку для тестовой выборки
f1 = f1_score(target_test, test_pred)

print('LogisticRegression Best F1-score:', grid_search_lr.best_score_)

In [ ]:
# Предсказываем классы на тестовой выборке
test_pred = grid_search_lr.predict(tf_idf_test)

# Выводим отчет по метрикам качества для тестовой выборки
report = metrics.classification_report(target_test, test_pred)
print(report)

# Вычисляем и выводим F1-оценку для тестовой выборки
f1 = metrics.f1_score(target_test, test_pred)
print(f"F1-оценка на тестовой выборке: {f1}")

# Вывод

Мы обучили две модели, логистическую регрессию и lightgbm, на данных для определения токсичности комментариев. С использованием кросс-валидации и поиска по сетке нашли оптимальные гиперпараметры для каждой модели.

На валидационной выборке получили F1-меру 0.768 для логистической регрессии и 0.658 для lightgbm. После этого мы проверили обе модели на тестовой выборке и получили F1-меру 0.768 для логистической регрессии и 0.658 для lightgbm.

Таким образом, мы можем заключить, что модель логистической регрессии лучше подходит для определения токсичности комментариев и построили модель со значением метрики качества F1 не меньше 0.75.